In [1]:
#include <vector>
#include <string>
#include <mutex>

class write_ahead_log{
public:
    void write(unsigned& index, std::string& str){
        std::unique_lock lock(mut);
        log.push_back(str);
    }
    bool read(unsigned& index, std::string& str){
        std::unique_lock lock(mut);
        if(index < log.size()) { str = log[index]; return true; }
        return false;
    }
private:
    std::vector<std::string> log;
    std::mutex mut;
};

In [1]:
class shared_state;

class state_changer{
friend class shared_state;
public:
    state_changer(shared_state& s):state(s){}
protected:
    virtual void on_save();
    virtual void on_load();
    virtual void on_change();
private:
    shared_state& state;
};

class shared_state{
friend class state_changer;
public:
};


In [1]:
#include <iostream>
#include <vector>
#include <sstream>

template <class T>
class scanner{
public:
	scanner(){}
protected:	
    virtual void on_scan(T&element) = 0;
	virtual void on_save(T&element, std::ostream&, bool scanned) = 0;
	virtual void on_load(T&element, std::istream&, bool scanned) = 0;
public:
	void scan() {
		for(auto& element: array){
			std::stringstream stream;
			on_save(element,stream,false);
			on_load(element,stream,false);

			on_scan(element);
			
			stream.clear();
			on_save(element,stream,true);
			on_load(element,stream,true);
		}
	}
	std::vector<T> array;
};

In [2]:
struct sqware_type{
	int N;
	int NxN;
}; 

class calc_sqware_scanner : public  scanner<sqware_type>{
public:
	calc_sqware_scanner():scanner(){}
private:
	virtual void  on_scan(sqware_type&e) override { e.NxN = e.N * e.N; }
	virtual void  on_save(sqware_type&e, std::ostream&out, bool scanned) override {
		if(scanned) out << e.NxN; else out << e.N;
	}
	virtual void  on_load(sqware_type&e, std::istream&in, bool scanned) override {
		if(scanned) in >> e.NxN; else in >> e.N;
	}
};

In [ ]:
{
	calc_sqware_scanner scanner_object;

	scanner_object.array.resize(10);
	for(int i=0;i<10;i++) scanner_object.array[i].N = i;

	scanner_object.scan();

	for(int i=0;i<10;i++)
		std::cout << scanner_object.array[i].N	<< " * " << scanner_object.array[i].N << " = "
			<< scanner_object.array[i].NxN << std::endl; 
}